In [10]:
import os
from dotenv import load_dotenv

from langchain_core.documents import Document
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_community.vectorstores.oraclevs import OracleVS

from langchain_community.vectorstores.utils import (
    DistanceStrategy,
)

import oracledb

In [11]:
_ = load_dotenv()
un = os.getenv("ORACLE_USERNAME")
pw = os.getenv("ORACLE_PASSWORD")
dsn = os.getenv("ORACLE_DSN")
config_dir = "/tmp/wallet"
wallet_location = "/tmp/wallet"
wallet_password = os.getenv("WALLET_PASSWORD")

compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("GENAI_ENDPOINT")

In [12]:
embeddings = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
)

In [18]:
texts_1 = [
    "人工知能は1956年に学問分野として設立されました。",
    "アラン・チューリングは人工知能の研究を大規模に行った最初の人物です。",
    "ロンドンのマイダ・ベールで生まれたチューリングは、イングランド南部で育ちました。",
]

texts_2 = [
    "機械学習は薬の設計に使用されています。",
    "AIアルゴリズムを用いた計算合成は分子の特性を予測します。",
    "DDR1は癌や線維症に関与しています。"
]

In [20]:
with oracledb.connect(user=un, password=pw, dsn=dsn, config_dir=config_dir, wallet_location=wallet_location, wallet_password=wallet_password) as connection:
    oracle_vs = OracleVS(
        client=connection,
        embedding_function=embeddings,
        table_name="DEMO",
        distance_strategy=DistanceStrategy.COSINE,
        query="What is a Oracle Database"
    )
    oracle_vs.add_texts(
        texts=texts_1,
        metadatas=[
            {"subject": "history"}
        ]
    )
    oracle_vs.add_texts(
        texts=texts_2,
        metadatas=[
            {"subject": "biology"}
        ]
    )